In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import plotly.graph_objects as go
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers

np.set_printoptions(precision=6, suppress=True)

In [ ]:
base_dir = "./data/planes"
data_dir = './images'

if not os.path.exists(data_dir):
    os.mkdir(data_dir)

train_dir = os.path.join(data_dir, 'train')    # katalog zbioru treningowego
valid_dir = os.path.join(data_dir, 'valid')    # katalog zbioru walidacyjnego
test_dir = os.path.join(data_dir, 'test')      # katalog zbioru testowego

train_fighter_dir = os.path.join(train_dir, 'fighter-jet')
train_rocket_dir = os.path.join(train_dir, 'rocket')
train_missile_dir = os.path.join(train_dir, 'missile')

valid_fighter_dir = os.path.join(valid_dir, 'fighter-jet')
valid_rocket_dir = os.path.join(valid_dir, 'rocket')
valid_missile_dir = os.path.join(valid_dir, 'missile')

test_fighter_dir = os.path.join(test_dir, 'fighter-jet')
test_rocket_dir = os.path.join(test_dir, 'rocket')
test_missile_dir = os.path.join(test_dir, 'missile')

for directory in (train_dir, valid_dir, test_dir):
    if not os.path.exists(directory):
        os.mkdir(directory)

dirs = [train_fighter_dir, train_rocket_dir, train_missile_dir, valid_fighter_dir, valid_rocket_dir, valid_missile_dir, test_fighter_dir, test_rocket_dir, test_missile_dir]

for dir in dirs:
    if not os.path.exists(dir):
        os.mkdir(dir)

In [ ]:
fighter_fnames = os.listdir(os.path.join(base_dir, 'fighter-jet'))
rocket_fnames = os.listdir(os.path.join(base_dir, 'rocket'))
missile_fnames = os.listdir(os.path.join(base_dir, 'missile'))

fighter_fnames = [fname for fname in fighter_fnames if fname.split('.')[1].lower() in ['jpg', 'png', 'jpeg']]
rocket_fnames = [fname for fname in rocket_fnames if fname.split('.')[1].lower() in ['jpg', 'png', 'jpeg']]
missile_fnames = [fname for fname in missile_fnames if fname.split('.')[1].lower() in ['jpg', 'png', 'jpeg']]

In [ ]:
print(f"Fighters: {len(fighter_fnames)}")
print(f"Rockets: {len(rocket_fnames)}")
print(f"Missiles: {len(missile_fnames)}")

In [ ]:
size = min(len(fighter_fnames), len(rocket_fnames), len(missile_fnames))

train_size = int(np.floor(0.65 * size))
valid_size = int(np.floor(0.2 * size))
test_size = size - train_size - valid_size

train_idx = train_size
valid_idx = train_size + valid_size
test_idx = train_size + valid_size + test_size

In [ ]:
for i, fname in enumerate(fighter_fnames):
    if i <= train_idx:
        src = os.path.join(base_dir, 'fighter-jet', fname)
        dst = os.path.join(train_fighter_dir, fname)
        shutil.copyfile(src, dst)
    elif train_idx < i <= valid_idx:
        src = os.path.join(base_dir, 'fighter-jet', fname)
        dst = os.path.join(valid_fighter_dir, fname)
        shutil.copyfile(src, dst)
    elif valid_idx < i < test_idx:
        src = os.path.join(base_dir, 'fighter-jet', fname)
        dst = os.path.join(test_fighter_dir, fname)
        shutil.copyfile(src, dst)

for i, fname in enumerate(rocket_fnames):
    if i <= train_idx:
        src = os.path.join(base_dir, 'rocket', fname)
        dst = os.path.join(train_rocket_dir, fname)
        shutil.copyfile(src, dst)
    elif train_idx < i <= valid_idx:
        src = os.path.join(base_dir, 'rocket', fname)
        dst = os.path.join(valid_rocket_dir, fname)
        shutil.copyfile(src, dst)
    elif valid_idx < i < test_idx:
        src = os.path.join(base_dir, 'rocket', fname)
        dst = os.path.join(test_rocket_dir, fname)
        shutil.copyfile(src, dst)

for i, fname in enumerate(missile_fnames):
    if i <= train_idx:
        src = os.path.join(base_dir, 'missile', fname)
        dst = os.path.join(train_missile_dir, fname)
        shutil.copyfile(src, dst)
    elif train_idx < i <= valid_idx:
        src = os.path.join(base_dir, 'missile', fname)
        dst = os.path.join(valid_missile_dir, fname)
        shutil.copyfile(src, dst)
    elif valid_idx < i < test_idx:
        src = os.path.join(base_dir, 'missile', fname)
        dst = os.path.join(test_missile_dir, fname)
        shutil.copyfile(src, dst)

print('fighter jet - zbiór treningowy', len(os.listdir(train_fighter_dir)))
print('fighter jet - zbiór walidacyjny', len(os.listdir(valid_fighter_dir)))
print('fighter jet - zbiór testowy', len(os.listdir(test_fighter_dir)))

print('rocket - zbiór treningowy', len(os.listdir(train_rocket_dir)))
print('rocket - zbiór walidacyjny', len(os.listdir(valid_rocket_dir)))
print('rocket - zbiór testowy', len(os.listdir(test_rocket_dir)))

print('missile - zbiór treningowy', len(os.listdir(train_missile_dir)))
print('missile - zbiór walidacyjny', len(os.listdir(valid_missile_dir)))
print('missile - zbiór testowy', len(os.listdir(test_missile_dir)))

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=45,  # zakres kąta o który losowo zostanie wykonany obrót obrazów
    rescale=1. / 255.,
    width_shift_range=0.25,  # pionowe przekształcenia obrazu
    height_shift_range=0.25,  # poziome przekształcenia obrazu
    shear_range=0.25,  # zares losowego przycianania obrazu
    zoom_range=0.25,  # zakres losowego przybliżania obrazu
    horizontal_flip=True,  # losowe odbicie połowy obrazu w płaszczyźnie poziomej
    fill_mode='nearest'  # strategia wypełniania nowo utworzonych pikseli, któe mogą powstać w wyniku przekształceń
)

# przeskalowujemy wszystkie obrazy o współczynnik 1/255
valid_datagen = ImageDataGenerator(rescale=1. / 255.)

train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='categorical')

valid_generator = valid_datagen.flow_from_directory(directory=valid_dir,
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='categorical')

In [ ]:
def display_augmented_images(directory, idx):

    fnames = [os.path.join(directory, fname) for fname in os.listdir(directory)]
    img_path = fnames[idx]
    img = image.load_img(img_path, target_size=(150, 150))

    x = image.img_to_array(img)
    x = x.reshape((1, ) + x.shape)

    i = 1
    plt.figure(figsize=(16, 8))
    for batch in train_datagen.flow(x, batch_size=1):
        plt.subplot(3, 4, i)
        plt.grid(False)
        imgplot = plt.imshow(image.array_to_img(batch[0]))
        i += 1
        if i % 13 == 0:
            break

In [ ]:
idx = 1
display_augmented_images(train_fighter_dir, idx)

In [ ]:
idx = 111
display_augmented_images(train_rocket_dir, idx)

In [ ]:
idx = 1
display_augmented_images(train_missile_dir, idx)

In [ ]:
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
conv_base.summary()

In [ ]:
model = Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(units=256, activation='relu'))
model.add(layers.Dense(units=3, activation='softmax'))
model.summary()

conv_base.trainable = False

In [ ]:
model.compile(optimizer = optimizers.legacy.RMSprop(learning_rate=1e-3),
              loss = "categorical_crossentropy",
              metrics = ["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint


es = EarlyStopping(monitor="val_accuracy", mode="max", verbose=0, patience=3)
mc = ModelCheckpoint(filepath="best_weights.hdf5", monitor="val_accuracy", mode="max", verbose=1, save_best_only=True)

In [ ]:
batch_size = 32
steps_per_epoch = train_size // batch_size
validation_steps = valid_size // batch_size

In [ ]:
history = model.fit(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=15,    # 100
                    validation_data=valid_generator,
                    validation_steps=validation_steps,
                    callbacks=[es, mc])

In [ ]:
def plot_hist(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['accuracy'], name='accuracy', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_accuracy'], name='val_accuracy', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='Accuracy vs. Val Accuracy', xaxis_title='Epoki', yaxis_title='Accuracy', yaxis_type='log')
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['loss'], name='loss', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_loss'], name='val_loss', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='Loss vs. Val Loss', xaxis_title='Epoki', yaxis_title='Loss', yaxis_type='log')
    fig.show()

plot_hist(history)

In [ ]:
model.load_weights("best_weights.hdf5")

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255.)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

y_prob = model.predict(test_generator, test_generator.samples)
y_prob

In [ ]:
y_pred = np.argmax(y_prob, axis=1)
y_pred

In [ ]:
predictions  = pd.DataFrame({'class': y_pred})
predictions

In [ ]:
y_true = test_generator.classes
y_true

In [ ]:
y_pred = predictions['class'].values
y_pred

In [ ]:
test_generator.class_indices

In [ ]:
cm = confusion_matrix(y_true, y_pred)
cm

In [ ]:
print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))